# --- Day 11 Monkey in the Middle ---

https://adventofcode.com/2022/day/11

In [1]:
from collections import defaultdict

## Get Input Data

In [2]:
def get_data(filename):
    """Get input data for day 11 puzzle.
    
    Parameters
    ----------
    filename : str
    
    Returns
    -------
    monkey_notes : dict
        Dictionary of details about how each monkey handles items in your backpack.
    """
    monkey_notes = defaultdict(dict)

    with open(f'../inputs/{filename}.txt') as _file:
        for line in _file:
            line = line.rstrip()
            if line.startswith('Monkey'):
                ID = int(line[-2])  # There're only 8 monkeys in the input file
            elif line.startswith('  Starting items:'):
                monkey_notes[ID]['items'] = [int(x) for x in line[18:].split(',')]
            elif line.startswith('  Operation:'):
                monkey_notes[ID]['op'] = line[19:]
            elif line.startswith('  Test:'):
                monkey_notes[ID]['test'] = int(line[21:])
            elif line.startswith('    If true:'):
                monkey_notes[ID]['true'] = int(line[-1])
            elif line.startswith('    If false:'):
                monkey_notes[ID]['false'] = int(line[-1])

    return monkey_notes

In [3]:
test_monkey_notes = get_data('test_monkey_notes')
test_monkey_notes

defaultdict(dict,
            {0: {'items': [79, 98],
              'op': 'old * 19',
              'test': 23,
              'true': 2,
              'false': 3},
             1: {'items': [54, 65, 75, 74],
              'op': 'old + 6',
              'test': 19,
              'true': 2,
              'false': 0},
             2: {'items': [79, 60, 97],
              'op': 'old * old',
              'test': 13,
              'true': 1,
              'false': 3},
             3: {'items': [74],
              'op': 'old + 3',
              'test': 17,
              'true': 0,
              'false': 1}})

## Part 1
---

In [4]:
def pass_items_around(monkey_notes):
    """Simulate 20 rounds of monkeys inspecting items in your backpak and thowing
    them to another monkey.

    Parameters
    ----------
    monkey_notes : dict
        Contains details of 'worry level' for each item that a monkey has taken
        from your backpack.

    Returns
    -------
    monkey_business : int
        Measure of the product of the top two most productive monkeys 
        (those who have inspected the most items)
    """

    # Keep track of how many items each monkey has inspected
    inspected_counts = [0] * len(monkey_notes)
    
    for round in range(20):
        for m in range(len(monkey_notes)):
            for item in monkey_notes[m]['items']:
                inspected_counts[m] += 1

                old = item  # Use 'old', because that's what the op says
                new_worry_level = eval(monkey_notes[m]['op']) // 3

                if new_worry_level % monkey_notes[m]['test'] == 0:
                    next_monkey = monkey_notes[m]['true']
                else:
                    next_monkey = monkey_notes[m]['false']

                # Throw the item to the next monkey
                monkey_notes[next_monkey]['items'].append(new_worry_level)
            
            # Clear current monkey's items list
            monkey_notes[m]['items'] = []

    # Calculate "monkey business" metric
    top_two = sorted(inspected_counts)[-2:]
    monkey_business = top_two[0] * top_two[1]

    return monkey_business

### Run on Test Data

In [5]:
pass_items_around(get_data('test_monkey_notes')) == 10605

True

### Run on Input Data

In [6]:
pass_items_around(get_data('monkey_notes'))

55458

## Part 2
---

## Dan Ready to the Rescue!

I would have **NEVER** gotten this on my own.  
[https://github.com/dready10/AOC22/blob/master/day11.c](https://github.com/dready10/AOC22/blob/master/day11.c)

In [7]:
def get_data2(filename):
    """Get input data for day 11 puzzle.
    
    Parameters
    ----------
    filename : str
    
    Returns
    -------
    monkey_notes : dict
        Dictionary of details about how each monkey handles items in your backpack.
    """
    monkey_notes = defaultdict(dict)

    with open(f'../inputs/{filename}.txt') as _file:
        for line in _file:
            line = line.rstrip()
            if line.startswith('Monkey'):
                ID = int(line[-2])  # There're only 8 monkeys in the input file
            elif line.startswith('  Starting items:'):
                monkey_notes[ID]['items'] = [float(x) for x in line[18:].split(',')]
            elif line.startswith('  Operation:'):
                op_line = line[23:]
                if op_line == '* old':
                    monkey_notes[ID]['op'] = 'exp'
                    monkey_notes[ID]['operand'] = 2.0
                elif op_line.startswith('*'):
                    monkey_notes[ID]['op'] = 'mult'
                    monkey_notes[ID]['operand'] = float(op_line[2:])
                elif op_line.startswith('+'):
                    monkey_notes[ID]['op'] = 'add'
                    monkey_notes[ID]['operand'] = float(op_line[2:])
            elif line.startswith('  Test:'):
                monkey_notes[ID]['test'] = float(line[21:])
            elif line.startswith('    If true:'):
                monkey_notes[ID]['true'] = int(line[-1])
            elif line.startswith('    If false:'):
                monkey_notes[ID]['false'] = int(line[-1])

    return monkey_notes

In [8]:
import math

In [9]:
def pass_items_around2(monkey_notes, num_rounds):
    """Simulate 20 rounds of monkeys inspecting items in your backpak and thowing
    them to another monkey.

    Parameters
    ----------
    monkey_notes : dict
        Contains details of 'worry level' for each item that a monkey has taken
        from your backpack.

    Returns
    -------
    monkey_business : int
        Measure of the product of the top two most productive monkeys 
        (those who have inspected the most items)
    """

    # Keep track of how many items each monkey has inspected
    inspected_counts = [0] * len(monkey_notes)
    
    # THIS IS THE TRICK TO REDUCE THE WORRY LEVEL TO MANAGABLE SIZE!
    # Calculate the lowest common multiple (lcm) of all the 'test' values.
    # These are all primes, so the lcm is equal to the pruduct of them all.
    # Then, this can be used to scale the worry level because all the actual
    # tests are whether the worry level is divisible by the 'test value.
    # 
    # When scaling, we calculate the reduced_worry_level as the modulo of
    # new_worry_level / test_lcm.
    #
    # AND THEN, when the modulo of reduced_worry_level / test == 0, gives the 
    # same result as it would if the worry_level wasn't reduce.
    test_lcm = math.prod([monkey_notes[m]['test'] for m in range(len(monkey_notes))])

    for _ in range(num_rounds):
        for m in range(len(monkey_notes)):
            
            for item in monkey_notes[m]['items']:
                inspected_counts[m] += 1

                if monkey_notes[m]['op'] == 'exp':
                    new_worry_level = item ** monkey_notes[m]['operand']
                elif monkey_notes[m]['op'] == 'mult':
                    new_worry_level = item * monkey_notes[m]['operand']
                elif monkey_notes[m]['op'] == 'add':
                    new_worry_level = item + monkey_notes[m]['operand']

                # THIS IS THE TRICK TO REDUCE THE WORRY LEVEL TO MANAGABLE SIZE!
                # current_worry_level = new_worry_level // 3
                reduced_worry_level = new_worry_level % test_lcm

                if reduced_worry_level % monkey_notes[m]['test'] == 0:
                    next_monkey = monkey_notes[m]['true']
                else:
                    next_monkey = monkey_notes[m]['false']

                # Throw the item to the next monkey
                monkey_notes[next_monkey]['items'].append(reduced_worry_level)

            # Clear current monkey's items list
            monkey_notes[m]['items'] = []

    # Calculate "monkey business" metric
    top_two = sorted(inspected_counts)[-2:]
    monkey_business = top_two[0] * top_two[1]
    print(inspected_counts)

    return monkey_business

## Run on Test Data

In [10]:
pass_items_around2(get_data2('test_monkey_notes'), 10_000) == 2713310158

[52166, 47830, 1938, 52013]


True

In [11]:
pass_items_around2(get_data2('monkey_notes'), 10_000)

[120477, 5452, 16145, 109710, 16135, 113731, 109712, 120422]


14508081294

## How the `test_lcm` works
Work out a simple example with just two test values, which are prime: `13` and `17`.

In [12]:
lcm = 13 * 17
lcm

221

In [13]:
for i in range(1, 10):
    reduced13 = (13 ** i) % lcm
    modulo13 = reduced13 % 13  # Should be 0 for all values of reduced13

    reduced17 = (17 ** i) % lcm
    modulo17 = reduced17 % 17  # Should be 0 for all values of reduced17

    print('13: ', i, reduced13, modulo13)
    print('17: ', i, reduced17, modulo17)

13:  1 13 0
17:  1 17 0
13:  2 169 0
17:  2 68 0
13:  3 208 0
17:  3 51 0
13:  4 52 0
17:  4 204 0
13:  5 13 0
17:  5 153 0
13:  6 169 0
17:  6 170 0
13:  7 208 0
17:  7 17 0
13:  8 52 0
17:  8 68 0
13:  9 13 0
17:  9 51 0


# Second Failed Attempt!
I thought maybe I would just need to fiddle with the data type being used to store the `worry_level` values...

Nope!

That doesn't work...

In [14]:
# import numpy as np

In [15]:
# def get_data2(filename):
#     """Get input data for day 11 puzzle.
    
#     Parameters
#     ----------
#     filename : str
    
#     Returns
#     -------
#     monkey_notes : dict
#         Dictionary of details about how each monkey handles items in your backpack.
#     """
#     monkey_notes = defaultdict(dict)

#     with open(f'../inputs/{filename}.txt') as _file:
#         for line in _file:
#             line = line.rstrip()
#             if line.startswith('Monkey'):
#                 ID = int(line[-2])  # There're only 8 monkeys in the input file
#             elif line.startswith('  Starting items:'):
#                 items = [float(x) for x in line[18:].split(',')]
#                 monkey_notes[ID]['items'] = np.array(items, dtype=np.longdouble)
#                 # monkey_notes[ID]['item_factors'] = [find_prime_factors(x) for x in monkey_notes[ID]['items']]
#             elif line.startswith('  Operation:'):
#                 op_line = line[23:]
#                 if op_line == '* old':
#                     monkey_notes[ID]['op'] = 'exp'
#                     monkey_notes[ID]['operand'] = 2.0
#                 elif op_line.startswith('*'):
#                     monkey_notes[ID]['op'] = 'mult'
#                     monkey_notes[ID]['operand'] = float(op_line[2:])
#                 elif op_line.startswith('+'):
#                     monkey_notes[ID]['op'] = 'add'
#                     monkey_notes[ID]['operand'] = float(op_line[2:])
#             elif line.startswith('  Test:'):
#                 monkey_notes[ID]['test'] = float(line[21:])
#             elif line.startswith('    If true:'):
#                 monkey_notes[ID]['true'] = int(line[-1])
#             elif line.startswith('    If false:'):
#                 monkey_notes[ID]['false'] = int(line[-1])

#     return monkey_notes

In [16]:
# def pass_items_around3(monkey_notes, num_rounds):
#     """Simulate 20 rounds of monkeys inspecting items in your backpak and thowing
#     them to another monkey.

#     Parameters
#     ----------
#     monkey_notes : dict
#         Contains details of 'worry level' for each item that a monkey has taken
#         from your backpack.

#     Returns
#     -------
#     monkey_business : int
#         Measure of the product of the top two most productive monkeys 
#         (those who have inspected the most items)
#     """

#     # Keep track of how many items each monkey has inspected
#     inspected_counts = [0] * len(monkey_notes)
    
#     for _ in range(num_rounds):
#         for m in range(len(monkey_notes)):
            
#             # print('monkey ', m)
#             # print('before')
#             # print('monkey 0: ', monkey_notes[0]['items'])
#             # print('monkey 1: ', monkey_notes[1]['items'])
#             # print('monkey 2: ', monkey_notes[2]['items'])
#             # print('monkey 3: ', monkey_notes[3]['items'])

#             for item in monkey_notes[m]['items']:
#                 inspected_counts[m] += 1

#                 if monkey_notes[m]['op'] == 'exp':
#                     new_worry_level = item ** monkey_notes[m]['operand']
#                 elif monkey_notes[m]['op'] == 'mult':
#                     new_worry_level = item * monkey_notes[m]['operand']
#                 elif monkey_notes[m]['op'] == 'add':
#                     new_worry_level = item + monkey_notes[m]['operand']

#                 # current_worry_level = new_worry_level // 3
#                 current_worry_level = new_worry_level

#                 if current_worry_level % monkey_notes[m]['test'] == 0:
#                     next_monkey = monkey_notes[m]['true']
#                 else:
#                     next_monkey = monkey_notes[m]['false']

#                 # print(new_worry_level, monkey_notes[m]['test'], new_worry_level % monkey_notes[m]['test'], 
#                 # next_monkey, current_worry_level
#                 # )


#                 # Throw the item to the next monkey
#                 monkey_notes[next_monkey]['items'] = np.append(monkey_notes[next_monkey]['items'], current_worry_level)
#                 # print(monkey_notes[next_monkey]['items'])

#             # Clear current monkey's items list
#             monkey_notes[m]['items'] = np.array([], dtype=np.longdouble)

#             # print('after')
#             # print('monkey 0: ', monkey_notes[0]['items'])
#             # print('monkey 1: ', monkey_notes[1]['items'])
#             # print('monkey 2: ', monkey_notes[2]['items'])
#             # print('monkey 3: ', monkey_notes[3]['items'])
#             # print('\n')

#     # Calculate "monkey business" metric
#     top_two = sorted(inspected_counts)[-2:]
#     monkey_business = top_two[0] * top_two[1]
#     print(inspected_counts)

#     return monkey_business

# First Failed Attempt at Part 2
I initially went down the path of storing the large `worry_level` values as prime factors...  
That doesn't work.

In [17]:
# import math

In [18]:
# def find_prime_factors(n):
#     factors = []

#     while n % 2 == 0:
#         factors.append(2)
#         n /= 2
    
#     for i in range(3, int(math.sqrt(n))+1, 2):
#         while n % i == 0:
#             factors.append(i)
#             n /= i
    
#     if n > 2:
#         factors.append(int(n))

#     return factors

In [19]:
# def pass_items_around2(monkey_notes):
#     """Simulate 10,000(!) rounds of monkeys inspecting items in your backpak and throwing
#     them to another monkey.
    
#     In this calc, worry levels are NOT floor divided by 3 to keep them in check...

#     Parameters
#     ----------
#     monkey_notes : dict
#         Contains details of 'worry level' for each item that a monkey has taken from your backpack.
    
#     Returns
#     -------
#     monkey_business : int
#         Measure of the product of the top two most productive monkeys 
#         (those who have inspected the most items)
#     """
#     # Keep track of how many items each monkey has inspected
#     inspected_counts = [0] * len(monkey_notes)
    
#     for round in range(20):
#         for m in range(len(monkey_notes)):
#             for item in monkey_notes[m]['item_factors']:
#                 inspected_counts[m] += 1
                
#                 if monkey_notes[m]['op'] == '* old':
#                     # Double the prime factors
#                     item = item * 2
#                 elif monkey_notes[m]['op'][0] == '*':
#                     # Add the new factor to the list of prime factors (these are always prime)
#                     new_factor = int(monkey_notes[m]['op'][2:])
#                     item.append(new_factor)
#                 elif monkey_notes[m]['op'][0] == '+':
#                     add_value = int(monkey_notes[m]['op'][2:])
#                     big_num = math.prod(item) + add_value
#                     item = find_prime_factors(big_num)

#                 denom = monkey_notes[m]['test']

#                 if denom in item:
#                     next_monkey = monkey_notes[m]['true']
#                 else:
#                     next_monkey = monkey_notes[m]['false']
                
#                 # Throw the item to the next monkey
#                 monkey_notes[next_monkey]['item_factors'].append(item)
            
#             # Clear current monkey's items list
#             monkey_notes[m]['item_factors'] = []

#     # Calculate "monkey business" metric
#     top_two = sorted(inspected_counts)[-2:]
#     monkey_business = top_two[0] * top_two[1]
#     print(inspected_counts)

#     return monkey_businessa